<a href="https://colab.research.google.com/github/psameni/INFO5731_Spring2020/blob/master/INFO5731_PRANAYA_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import tweepy
import csv
import pandas as pd

#using twitter Authentication to connect to twitter api with twitter provided keys.
authentication = tweepy.OAuthHandler('Dpp7E4W9YkKGVszqezpjpSlLc', 'bczWmE17Ve1DJHKJY0Z9eQYuAupPZ63JfL9AGnkuX12SUZkCWJ')
authentication.set_access_token('4289066089-Vd6LpZcoLUERdoFJTnk3J12w8M2sgajQ8HDIlaL', 'yebOZX98i8RtrsMNcWrLBh54K6ypZXi3KejgXSzunyxnH')
api = tweepy.API(authentication,wait_on_rate_limit=True)

#file operation
csvFile = open('tagdata.csv', 'w')
data = pd.read_csv('tagdata.csv', names=['tweet_data'], header=None)
write_to_csv = csv.writer(csvFile)

#writing to file using csvwriter
for tweet_data in tweepy.Cursor(api.search,q="#wuhancoronavirus",lang="en").items(100):
  write_to_csv.writerow([tweet_data.text.encode('utf-8')])
#reading csv file 
data = pd.read_csv('tagdata.csv',error_bad_lines=False,names=['tweet_data'])
import string

#Removing noise (Punctuation,extra words)
data['tweet_data']=data['tweet_data'].str.replace('[{}]'.format(string.punctuation), '')
data['tweet_data']=data['tweet_data'].str.replace('[^A-Za-z ]','')
data['tweet_data']=data['tweet_data'].str.replace('bRT','')
# print(data['tweet_data'].head())

#Removing numbers
data['tweet_data'] = data['tweet_data'].str.replace('\d+', '')

#Removing StopWords
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop=stopwords.words('english')
data['tweet_data']=data['tweet_data'].apply(lambda x:" ".join(y for y in x.split() if y not in stop))


#Lower casing
data['tweet_data']=data['tweet_data'].str.lower()
data2=data["tweet_data"]
#stemming
from nltk.stem import PorterStemmer
st=PorterStemmer()
data['tweet_data']=data['tweet_data'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

#Lemmetization
from textblob import Word
import nltk
nltk.download('wordnet')
data['tweet_data']=data['tweet_data'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['tweet_data'].head(5)
#saving the formatted sentences to "Formatted_sentences.txt"
# data['tweet_data'].to_csv("Formatted_tweets.csv",header="Tweets",index=False)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0    rajeevsrinivasa karnataka enforc cordonsanitai...
1    nikkimiumiu a hongkong femal polic infect wuha...
2    allmui hkpolic use ruler measur distanc tabl b...
3    mymonke chinaliedpeopledi nnaround day ago bas...
4    rajeevsrinivasa karnataka enforc cordonsanitai...
Name: tweet_data, dtype: object

### Question 1: Understand N-gram
(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the noun phrases and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets).

In [23]:
# (1) Count the frequency of all the N-grams (N=3).

from textblob import Word,TextBlob

import nltk
nltk.download('wordnet')#ngram features
nltk.download('punkt')
for i in range(len(data["tweet_data"])):
 print(TextBlob(data["tweet_data"][i]).ngrams(3))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[WordList(['rajeevsrinivasa', 'karnataka', 'enforc']), WordList(['karnataka', 'enforc', 'cordonsanitair']), WordList(['enforc', 'cordonsanitair', 'around']), WordList(['cordonsanitair', 'around', 'wuhancoronaviru']), WordList(['around', 'wuhancoronaviru', 'hit']), WordList(['wuhancoronaviru', 'hit', 'kasaragod']), WordList(['hit', 'kasaragod', 'klnnkejriw']), WordList(['kasaragod', 'klnnkejriw', 'broke']), WordList(['klnnkejriw', 'broke', 'cordxexxa'])]
[WordList(['nikkimiumiu', 'a', 'hongkong']), WordList(['a', 'hongkong', 'femal']), WordList(['hongkong', 'femal', 'polic']), WordList(['femal', 'polic', 'infect']), WordList(['polic', 'infect', 'wuhancoronaviru']), WordList(['infect', 'wuhancoronaviru', 'reject']), WordList(['wuhancoronaviru', 'reject', 'wear']), W

In [24]:
# (2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

import nltk

bigrams_lst=[]
tokens_lst=[]
for i in range(len(data["tweet_data"])):
 nltk_tokens = nltk.word_tokenize(data["tweet_data"][i])  
 tokens_lst= tokens_lst+(nltk_tokens)
 bigrams_lst=bigrams_lst+(list(nltk.bigrams(nltk_tokens)))
bigrams_dict={}
for  i in range(len(bigrams_lst)):
    bigrams_dict[bigrams_lst[i]]=bigrams_lst.count(bigrams_lst[i])/tokens_lst.count(bigrams_lst[i][1])
print(bigrams_dict)

{('rajeevsrinivasa', 'karnataka'): 1.0, ('karnataka', 'enforc'): 1.0, ('enforc', 'cordonsanitair'): 1.0, ('cordonsanitair', 'around'): 1.0, ('around', 'wuhancoronaviru'): 0.28125, ('wuhancoronaviru', 'hit'): 1.0, ('hit', 'kasaragod'): 1.0, ('kasaragod', 'klnnkejriw'): 1.0, ('klnnkejriw', 'broke'): 1.0, ('broke', 'cordxexxa'): 1.0, ('nikkimiumiu', 'a'): 0.75, ('a', 'hongkong'): 1.0, ('hongkong', 'femal'): 1.0, ('femal', 'polic'): 1.0, ('polic', 'infect'): 0.75, ('infect', 'wuhancoronaviru'): 0.125, ('wuhancoronaviru', 'reject'): 1.0, ('reject', 'wear'): 1.0, ('wear', 'mask'): 1.0, ('mask', 'carita'): 1.0, ('carita', 'medic'): 1.0, ('medic', 'centr'): 1.0, ('centr', 'she'): 1.0, ('she', 'cxexxa'): 1.0, ('allmui', 'hkpolic'): 0.5, ('hkpolic', 'use'): 0.23529411764705882, ('use', 'ruler'): 1.0, ('ruler', 'measur'): 1.0, ('measur', 'distanc'): 0.5, ('distanc', 'tabl'): 1.0, ('tabl', 'barnni'): 1.0, ('barnni', 'think'): 1.0, ('think', 'hkpolic'): 0.5, ('hkpolic', 'workplac'): 1.0, ('workplac

In [25]:
# (3) Extract all the noun phrases and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets).
import pandas as pd
import csv
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize
nouns_lst=[]
data_nouns={}
for i in data["tweet_data"]:
    words=word_tokenize(i)
    nouns_data=list(filter(lambda x : x[1]=="NN" ,nltk.pos_tag(words)))
#     print(nouns_data)
    values=list(map(lambda x: x[0], nouns_data))
    data_nouns[i]=values
    nouns_lst=nouns_lst+values
nouns_set=set(nouns_lst)
# print(nouns_lst)
df = pd.DataFrame(index=data["tweet_data"],columns=nouns_set)


for i in range(len(df)):
    for noun in nouns_set:
      df.at[data["tweet_data"][i],noun]=(data_nouns[data["tweet_data"][i]].count(noun))/(nouns_lst.count(noun))
print(df)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
                                                   wuhancoronaviru  ... countri
tweet_data                                                          ...        
rajeevsrinivasa karnataka enforc cordonsanitair...       0.0357143  ...       0
nikkimiumiu a hongkong femal polic infect wuhan...       0.0357143  ...       0
allmui hkpolic use ruler measur distanc tabl ba...               0  ...       0
mymonke chinaliedpeopledi nnaround day ago basi...               0  ...       0
rajeevsrinivasa karnataka enforc cordonsanitair...       0.0357143  ...       0
...                                                            ...  ...     ...
bevid china sponsor china must held account spr...               0  ...       0
roofmonqey ahhh hey china death toll wuhancoron...       0.0357143  ...       0
mymonke chin

### Question 2: Undersand TF-IDF and Document representation
(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program:

(1) To build the documents-terms weights (tf*idf) matrix bold text.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using cosine similarity.

In [26]:
#(1) To build the documents-terms weights (tf*idf) matrix bold text.
import math

tfidf_words_lst=[]
tfidf_data={}
for i in data["tweet_data"]:
    words=word_tokenize(i)
    nouns_data=list(nltk.pos_tag(words))
#     print(nouns_data)
    values=list(map(lambda x: x[0], nouns_data))
    tfidf_data[i]=values
    nouns_lst=nouns_lst+values
nouns_set=set(nouns_lst)

tdidf = pd.DataFrame(index=data["tweet_data"],columns=nouns_set)

for i in range(len(df)):
    for noun in nouns_set:
      tdidf.at[data["tweet_data"][i],noun]=(tfidf_data[data["tweet_data"][i]].count(noun))*(math.log(len(data["tweet_data"])/(nouns_lst.count(noun))))
print(df)

                                                   wuhancoronaviru  ... countri
tweet_data                                                          ...        
rajeevsrinivasa karnataka enforc cordonsanitair...       0.0357143  ...       0
nikkimiumiu a hongkong femal polic infect wuhan...       0.0357143  ...       0
allmui hkpolic use ruler measur distanc tabl ba...               0  ...       0
mymonke chinaliedpeopledi nnaround day ago basi...               0  ...       0
rajeevsrinivasa karnataka enforc cordonsanitair...       0.0357143  ...       0
...                                                            ...  ...     ...
bevid china sponsor china must held account spr...               0  ...       0
roofmonqey ahhh hey china death toll wuhancoron...       0.0357143  ...       0
mymonke chinaliedpeopledi nnaround day ago basi...               0  ...       0
therealcook now that immun systemnnwuhancoronav...               0  ...       0
hyltonrobin xfxfxaxa break fox busi netw

In [28]:
#(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using cosine similarity.
import re
from collections import Counter

WORD = re.compile(r"\w+")


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

queryText = "This text is to test the cosine similarity witht the tweets data"
cosine_lst=[]
for tweetText in data["tweet_data"]:
    vector1 = text_to_vector(tweetText)
    vector2 = text_to_vector(queryText)
    cosine = get_cosine(vector1, vector2)
    cosine_lst.append(cosine)
print("Cosine similarity for the given text with the tweets data:")
print(cosine_lst)

Cosine similarity for the given text with the tweets data:
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1889822365046136, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15430334996209194, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14824986333222026, 0.0, 0.0, 0.0, 0.0, 0.14285714285714285, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1091089451179962, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.12262786789699315, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2672612419124244, 0.0, 0.16116459280507606, 0.0, 0.0, 0.15430334996209194, 0.0, 0.0, 0.0, 0.16116459280507606, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


### Question 3: Create your own training and evaluation data for sentiment analysis
(15 points). You dodn't need to write program for this question! Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification.

In [0]:
import re 
from textblob import TextBlob 

def clean_tweet(tweet): 
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 
  
def get_tweet_sentiment(tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'
filePointer = open("sentimetn_analysis_file.txt","a")
                               
sentiment_data={}  
counter=1
for tweet in data["tweet_data"]:
#     sentiment_data[tweet]=get_tweet_sentiment(tweet)
    sentiment_data=str(counter)+","+str(tweet)+","+str(get_tweet_sentiment(tweet))+"\n"
    filePointer.write(sentiment_data)
    counter=counter+1

Sentiment Analysis Test Data Github link :
https://github.com/psameni/Pranaya_INFO5731_Spring2020/blob/master/sentimetn_analysis_file.txt